<a href="https://colab.research.google.com/github/Felix-Eger/Rev-Engineering-Synthesizer/blob/main/Image_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 50 kB/s 
     |████████████████████████████████| 199 kB 55.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=d8631b7ba1039312e872fb6299c0b81c16b33fb1aa4c261bdd36ed919ed1bd9f
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
import pandas as pd
import numpy as np 
import librosa
import os
import pyspark
from pyspark.sql import SparkSession
from pathlib import Path
import matplotlib.pyplot as plt
import librosa.display

In [ ]:
def save_spectrogram(audio_fname):
    '''takes a path of an audio file and returns the mel spectrogram in relation to Hz and key'''
    
    #main_path = r"D:\Audio-Analysis\Audio-files\Presets_original"
    main_path = r"/content/drive/MyDrive/Big Data/Audio/Audio_original"
    
    com_path = main_path + '/' + audio_fname
    
    y, sr = librosa.load(com_path, sr=None)
    
    yt, index = librosa.effects.trim(y)

    hl = 512 # number of samples per time-step in spectrogram
    hi = 128 # Height of image
    wi = 384 # Width of image

    window = yt[0:wi*hl]
    S = librosa.feature.melspectrogram(y=window, sr=sr, n_mels=hi, fmax=8000, hop_length=hl)

    fig, ax = plt.subplots()

    S_dB = librosa.power_to_db(S, ref=np.max)
    img = librosa.display.specshow(S_dB)#, x_axis='time', y_axis='mel', sr=sr, fmax=8000, ax=ax)
    #fig.colorbar(img)#, ax=ax, format='%+2.0f dB')
    
    image_fname = audio_fname[:-4] 

    image_dir_path = r"/content/drive/MyDrive/Big Data/Images/Images_original/Mel_Spec"
    
    
    if image_dir_path:
        image_fname = Path(image_dir_path).as_posix() + '/' + image_fname
    if Path(image_fname).exists(): 
        return None
    
    try:
        plt.savefig(image_fname, dpi=100)
    except ValueError as verr:
        print('Failed to process %s %s' % (image_fname, verr))    

    return None

In [ ]:
def save_mfcc(audio_fname):
    '''takes a path of an audio file and returns the mel spectrogram in relation to Hz and key'''
    
    #main_path = r"D:\Audio-Analysis\Audio-files\Presets_original"
    main_path = r"/content/drive/MyDrive/Big Data/Audio/Audio_original"
    
    com_path = main_path + '/' + audio_fname
    
    y, sr = librosa.load(com_path, sr=None)
    
    yt, index = librosa.effects.trim(y)

    hl = 512 # number of samples per time-step in spectrogram
    hi = 128 # Height of image
    wi = 384 # Width of image

    window = yt[0:wi*hl]
    
    S = librosa.feature.melspectrogram(y=window, sr=sr, n_mels=hi, fmax=8000, hop_length=hl)
    m_htk = librosa.feature.mfcc(S=librosa.power_to_db(S), dct_type=3)
    
    fig, ax = plt.subplots() 
    img = librosa.display.specshow(m_htk)
    #fig.colorbar(img)
    
    image_fname = audio_fname[:-4] 

    image_dir_path = r"/content/drive/MyDrive/Big Data/Images/Images_original/MFCC"
    
    
    if image_dir_path:
        image_fname = Path(image_dir_path).as_posix() + '/' + image_fname
    if Path(image_fname).exists(): 
        return None
    
    try:
        plt.savefig(image_fname, dpi=100)
    except ValueError as verr:
        print('Failed to process %s %s' % (image_fname, verr))    

    return None

In [ ]:
aud_dir = r"/content/drive/MyDrive/Big Data/Audio/Audio_original"

In [ ]:
file_names = [name for name in os.listdir(aud_dir)]

In [ ]:
len(file_names)

1461

In [ ]:
from pyspark.sql import SparkSession
#spark = spark = SparkSession.builder().master("local[1]").appName("SparkByExamples.com").getOrCreate()

spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()

In [ ]:
rdd = spark.sparkContext.parallelize(file_names)

In [ ]:
rdd3=rdd.map(lambda x: save_mfcc(x))

rdd_final3 = rdd3.collect()